In [ ]:
import csv
import cv2
import numpy as np
import random
import pandas as pd
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer, KNNImputer
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GroupKFold
from sklearn.linear_model import LogisticRegression, LinearRegression
import os
from sklearn.decomposition import PCA
import math
import joblib

In [ ]:
x_test = pd.read_csv(f'/kaggle/input/tabular-playground-series-aug-2022/test.csv')
x_test = x_test.reset_index().set_index('id').drop('index', axis=1)

# create missing value flag
missing_measurement3_list = []
missing_measurement5_list = []
for element in x_test['measurement_3']:
    if np.isnan(element): #element is empty
        missing_measurement3_list.append(0) 
    else:
        missing_measurement3_list.append(1)
x_test['missing_measurement3'] = missing_measurement3_list
for element in x_test['measurement_5']:
    if np.isnan(element): #element is empty
        missing_measurement5_list.append(0) 
    else:
        missing_measurement5_list.append(1)
x_test['missing_measurement5'] = missing_measurement5_list


# fill the missing values
features = []
for f in x_test.columns:
    if f == 'loading' or (f.startswith('measurement') and f != 'measurement_17'):
        features.append(f)
imputer = SimpleImputer(strategy='most_frequent')
imputer.fit_transform(x_test[features])
x_test[features] = imputer.transform(x_test[features])

# find the correlation among measurement17 and measurement3~9           
for product in "F", "G", "H", "I":
    lr = LinearRegression().fit(x_test.loc[(x_test['product_code'] == product) & (x_test.measurement_17.isna() == False), ['measurement_3', 'measurement_4', 'measurement_5','measurement_6', 'measurement_7', 'measurement_8', 'measurement_9']],
                                x_test.loc[(x_test['product_code'] == product) & (x_test.measurement_17.isna() == False), ['measurement_17']])
    x_test.loc[(x_test['product_code'] == product), ['pred_m_17']] = lr.predict(x_test.loc[(x_test['product_code'] == product), ['measurement_3', 'measurement_4', 'measurement_5','measurement_6', 'measurement_7', 'measurement_8', 'measurement_9']])
    for index, row in x_test.iterrows():
        if row["product_code"] == product and math.isnan(row["measurement_17"]):
            x_test.loc[index, 'measurement_17'] = x_test.loc[index, 'pred_m_17']

# create new features
x_test['area'] = x_test['attribute_2'] * x_test['attribute_3']
x_test['average'] = x_test[[f for f in x_test.columns if f.startswith('measurement') and f != 'measurement_17' and f != 'measurement_0' and f != 'measurement_1' and f != 'measurement_2']].mean(axis=1)

# drop the column that is useless
x_test = x_test.drop([f for f in x_test.columns if f.startswith('measurement') and f != 'measurement_17' and f != 'measurement_0' and f != 'measurement_1' and f != 'measurement_2'], axis=1)


# standardize
x_test[['loading', 'measurement_17', 'measurement_0', 'measurement_1','measurement_2',
          'missing_measurement3', 'missing_measurement5']] = StandardScaler().fit_transform(x_test[['loading',
          'measurement_17', 'measurement_0', 'measurement_1', 'measurement_2', 'missing_measurement3', 'missing_measurement5']])
x_test[['area', 'average']] = StandardScaler().fit_transform(x_test[['area', 'average']])

# drop the column that is useless
x_test = x_test.drop(['product_code', 'attribute_3', 'attribute_2', 'attribute_1', 'attribute_0'], axis=1)

In [ ]:
# store prediction into submission.csv
test_data = []
loaded_model = joblib.load('model')
test_pred_list = loaded_model.predict_proba(x_test)[:,1]
with open(f'/kaggle/input/tabular-playground-series-aug-2022/sample_submission.csv', newline='') as csvfile:
    for row in csv.reader(csvfile, delimiter=','):
        test_data.append(row)
file = open('submission.csv', 'w', newline='')
csv_writer = csv.writer(file)

for i in range(len(test_data)):
    if i == 0:
        continue
    csv_writer.writerow([int(test_data[i][0]), test_pred_list[i-1]])